## Mounting the google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

## Importing the essential libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import tensorflow as tf
tf.__version__
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import tensorflow_hub as hub
import tensorflow_text as text


## Importing the dataset

In [4]:
dataset_train = pd.read_csv('drive/MyDrive/emails.csv',nrows=10000)
dataset_train.duplicated().sum()

0

In [5]:
def parse_raw_message(raw_message):
    lines = raw_message.split('\n')
    email = {}
    message = ''
    keys_to_extract = ['from', 'to']
    for line in lines:
        if ':' not in line:
            message += line.strip()
            email['body'] = message
        else:
            pairs = line.split(':')
            key = pairs[0].lower()
            val = pairs[1].strip()
            if key in keys_to_extract:
                email[key] = val
    return email

def parse_into_emails(messages):
    emails = [parse_raw_message(message) for message in messages]
    return {
        'body': map_to_list(emails, 'body'),
        'to': map_to_list(emails, 'to'),
        'from_': map_to_list(emails, 'from')
    }

def map_to_list(emails, key):
    results = []
    for email in emails:
        if key not in email:
            results.append('')
        else:
            results.append(email[key])
    return results

In [6]:
email_df = pd.DataFrame(parse_into_emails(dataset_train.message))
email_df.drop(email_df.query("body == '' | to == '' | from_ == ''").index, inplace=True)

In [7]:
email_df.head()

,body,to,from_
0,Here is our forecast,tim.belden@enron.com,phillip.allen@enron.com
1,Traveling to have a business meeting takes the...,john.lavorato@enron.com,phillip.allen@enron.com
2,test successful. way to go!!!,leah.arsdall@enron.com,phillip.allen@enron.com
3,"Randy,Can you send me a schedule of the salary...",randall.gay@enron.com,phillip.allen@enron.com
5,"Greg,How about either next Tuesday or Thursday...",greg.piper@enron.com,phillip.allen@enron.com


## Getting the shape of the dataset

In [8]:
email_df.shape

(9464, 3)

## Preprocessing the dataset

In [9]:
def preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    '''
    Preprocess a string.
    :parameter
        :param text: string - name of column containing text
        :param lst_stopwords: list - list of stopwords to remove
        :param flg_stemm: bool - whether stemming is to be applied
        :param flg_lemm: bool - whether lemmitisation is to be applied
    :return
        cleaned text
    '''

    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())

    ## Tokenize (convert from string to list)
    lst_text = text.split()

    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in lst_stopwords]

    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]

    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]

    ## back to string from list
    text = " ".join(lst_text)
    return text

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
lst_stopwords = nltk.corpus.stopwords.words("english")

In [12]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Cleaning the text over here

In [13]:
email_df["body_clean"] = email_df["body"].apply(lambda x: preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))
email_df.head()

,body,to,from_,body_clean
0,Here is our forecast,tim.belden@enron.com,phillip.allen@enron.com,forecast
1,Traveling to have a business meeting takes the...,john.lavorato@enron.com,phillip.allen@enron.com,traveling business meeting take fun trip espec...
2,test successful. way to go!!!,leah.arsdall@enron.com,phillip.allen@enron.com,test successful way go
3,"Randy,Can you send me a schedule of the salary...",randall.gay@enron.com,phillip.allen@enron.com,randycan send schedule salary level everyone t...
5,"Greg,How about either next Tuesday or Thursday...",greg.piper@enron.com,phillip.allen@enron.com,greghow either next tuesday thursdayphillip


In [14]:
email_df.groupby('to').describe()

body         \
                                                   count unique   
to                                                                
"'Arnold, John'" <john.arnold@enron.com>               9      3   
"'Eric Bass'" <Eric.Bass@enron.com>                    1      1   
"'Eric_Bass@enron.com'" <Eric_Bass@enron.com>          2      2   
"'John.Arnold@enron.com'" <John.Arnold@enron.com>      3      1   
"'K. Bass'" <daphneco64@bigplanet.com>                 1      1   
...                                                  ...    ...   
ywang@enron.com                                        6      2   
ywang@enron.com, patti.sullivan@enron.com, phil...    18      2   
zalaywan@caiso.com                                     3      1   
zalaywan@caiso.com, eschmid@caiso.com, crobinso...     3      1   
zeu3rk@msn.com                                         1      1   

                                                                                                       \
                                                                                                  top   
to                                                                                                      
"'Arnold, John'" <john.arnold@enron.com>            personal business.PMJohn -I completely underst...   
"'Eric Bass'" <Eric.Bass@enron.com>                 I hope you were only thinking good things!I ha...   
"'Eric_Bass@enron.com'" <Eric_Bass@enron.com>       It only takes about 10 minutes to get to highl...   
"'John.Arnold@enron.com'" <John.Arnold@enron.com>   I will not be able to attend the meeting but d...   
"'K. Bass'" <daphneco64@bigplanet.com>              I think we are going to stay in town and meet ...   
...                                                                                               ...   
ywang@enron.com                                               please add mike grigsby to distribution   
ywang@enron.com, patti.sullivan@enron.com, phil...  jane.m.tholt@enron.com, mike.grigsby@enron.com...   
zalaywan@caiso.com                                  put this into the congestion redesign file if ...   
zalaywan@caiso.com, eschmid@caiso.com, crobinso...  mary.hain@enron.comBelow is a summary of my SW...   
zeu3rk@msn.com                                      Got debt? We can help using Debt Consolidation...   

                                                        from_         \
                                                   freq count unique   
to                                                                     
"'Arnold, John'" <john.arnold@enron.com>              3     9      2   
"'Eric Bass'" <Eric.Bass@enron.com>                   1     1      1   
"'Eric_Bass@enron.com'" <Eric_Bass@enron.com>         1     2      1   
"'John.Arnold@enron.com'" <John.Arnold@enron.com>     3     3      1   
"'K. Bass'" <daphneco64@bigplanet.com>                1     1      1   
...                                                 ...   ...    ...   
ywang@enron.com                                       3     6      1   
ywang@enron.com, patti.sullivan@enron.com, phil...   15    18      1   
zalaywan@caiso.com                                    3     3      1   
zalaywan@caiso.com, eschmid@caiso.com, crobinso...    3     3      1   
zeu3rk@msn.com                                        1     1      1   

                                                                                            \
                                                                                       top   
to                                                                                           
"'Arnold, John'" <john.arnold@enron.com>                       John.Arnold@enron.com [SMTP   
"'Eric Bass'" <Eric.Bass@enron.com>                                    eric.bass@enron.com   
"'Eric_Bass@enron.com'" <Eric_Bass@enron.com>                          eric.bass@enron.com   
"'John.Arnold@enron.com'" <John.Arnold@enron.com>                    

In [15]:
email_df.groupby('from_').describe()

body         \
                                                   count unique   
from_                                                             
"ALLYSON FELLER, BLOOMBERG/ NEW YORK" <AFELLER@...     3      1   
"ALLYSON FELLER, BLOOMBERG/ NEW YORK" <AFELLER@...     1      1   
"Adrian Clark" <AClark@firstcallassociates.com>...     1      1   
"Allen, Phillip K." <Phillip.K.Allen@ENRON.com>        1      1   
"Andy Colman" <andy@spectronenergy.com>@ENRON          1      1   
...                                                  ...    ...   
wtashnek@aol.com                                       1      1   
yahoo-delivers@yahoo-inc.com                           8      4   
yevgeny.frolov@enron.com                               1      1   
yild@zdemail.zdlists.com                               3      1   
zachary.streight@enron.com                             1      1   

                                                                                                       \
                                                                                                  top   
from_                                                                                                   
"ALLYSON FELLER, BLOOMBERG/ NEW YORK" <AFELLER@...  Please send it to me.johnJohn,We are trying to...   
"ALLYSON FELLER, BLOOMBERG/ NEW YORK" <AFELLER@...  Please send it to me.johnJohn,We are trying to...   
"Adrian Clark" <AClark@firstcallassociates.com>...  I know a set up when I see one-----Original Me...   
"Allen, Phillip K." <Phillip.K.Allen@ENRON.com>     Ceci,I would like to stay current on listings ...   
"Andy Colman" <andy@spectronenergy.com>@ENRON       not mad at you-----Original Message-----Jon, c...   
...                                                                                               ...   
wtashnek@aol.com                                    I'm just checking to see if you got my e-mail ...   
yahoo-delivers@yahoo-inc.com                        [IMAGE]Yahoo! sent this email to you because y...   
yevgeny.frolov@enron.com                            kirk.mcdaniel@enron.comkirk.mcdaniel@enron.com...   
yild@zdemail.zdlists.com                            as links, visit the Yahoo! Internet Life home ...   
zachary.streight@enron.com                          John,I have attached a link to the first itera...   

                                                           to         \
                                                   freq count unique   
from_                                                                  
"ALLYSON FELLER, BLOOMBERG/ NEW YORK" <AFELLER@...    3     3      1   
"ALLYSON FELLER, BLOOMBERG/ NEW YORK" <AFELLER@...    1     1      1   
"Adrian Clark" <AClark@firstcallassociates.com>...    1     1      1   
"Allen, Phillip K." <Phillip.K.Allen@ENRON.com>       1     1      1   
"Andy Colman" <andy@spectronenergy.com>@ENRON         1     1      1   
...                                                 ...   ...    ...   
wtashnek@aol.com                                      1     1      1   
yahoo-delivers@yahoo-inc.com                          3     8      2   
yevgeny.frolov@enron.com                              1     1      1   
yild@zdemail.zdlists.com                              3     3      1   
zachary.streight@enron.com                            1     1      1   

                                                                                                       \
                                                                                                  top   
from_                                                                                                   
"ALLYSON FELLER, BLOOMBERG/ NEW YORK" <AFELLER@...                                  JPECHER@ENRON.COM   
"ALLYSON FELLER, BLOOMBERG/ NEW YORK" <AFELLER@...                                  JPECHER@ENRON.COM   
"Adrian Clark" <AClark@firstcallassociates.com>...                                        Maggi, Mike   
"Al

In [16]:
email_df.groupby('body_clean').describe()

body         \
                                                   count unique   
body_clean                                                        
                                                       9      4   
001nevereditthisline002this advertisement sent ...     1      1   
01 attachment free virus scan mail forwarded ph...     4      1   
010101 herald actual millenium unique date repe...     3      1   
072701francishere file need excel thank helpphi...     1      1   
...                                                  ...    ...   
zf trading 4                                           4      1   
zf wow would thunk prompt gas 6 zf aswide last ...     3      1   
zip250mb usb drive take anywhere_______________...     1      1   
zip250mb usb drive take anywhere_______________...     1      1   
zlaziccaisocom rabernathycaisocom pmackincaisoc...     3      1   

                                                                                                       \
                                                                                                  top   
body_clean                                                                                              
                                                                       AM ---------------------------   
001nevereditthisline002this advertisement sent ...  --001nevereditthisline002---------------------...   
01 attachment free virus scan mail forwarded ph...  /01 Attachment is free from viruses. Scan Mail...   
010101 herald actual millenium unique date repe...  01.01.01 heralds the actual millenium. A uniqu...   
072701francishere file need excel thank helpphi...  of 07/27/01Francis,Here are the files we need ...   
...                                                                                               ...   
zf trading 4                                                                            z/f trading 4   
zf wow would thunk prompt gas 6 zf aswide last ...  Z/F !!!!   wow.    Who would have thunk it.  P...   
zip250mb usb drive take anywhere_______________...  ==============================================...   
zip250mb usb drive take anywhere_______________...  ==============================================...   
zlaziccaisocom rabernathycaisocom pmackincaisoc...  zlazic@caiso.com, rabernathy@caiso.com, pmacki...   

                                                           to         \
                                                   freq count unique   
body_clean                                                             
                                                      4     9      4   
001nevereditthisline002this advertisement sent ...    1     1      1   
01 attachment free virus scan mail forwarded ph...    4     4      1   
010101 herald actual millenium unique date repe...    3     3      1   
072701francishere file need excel thank helpphi...    1     1      1   
...                                                 ...   ...    ...   
zf trading 4                                          4     4      1   
zf wow would thunk prompt gas 6 zf aswide last ...    3     3      1   
zip250mb usb drive take anywhere_______________...    1     1      1   
zip250mb usb drive take anywhere_______________...    1     1      1   
zlaziccaisocom rabernathycaisocom pmackincaisoc...    3     3      1   

                                                                                                       \
                                                                                                  top   
body_clean                                                                                              
                                                                              John Arnold/HOU/ECT@ECT   
001nevereditthisline002this advertisement sent ...                              jarnold@ect.enron.com   
01 attachment free virus scan mail forwarded ph...                                Anne.Bike@enron.com   
010

## Creating the dependent variable over here

In [17]:
fraud_related_words = ['fraud', 'scam', 'phishing', 'malware', 'deception', 'hoax']  # Add more fraud-related words if needed

email_df['fraud'] = email_df['body_clean'].apply(lambda x: 1 if any(word in x.lower() for word in fraud_related_words) else 0)
email_df.head()

,body,to,from_,body_clean,fraud
0,Here is our forecast,tim.belden@enron.com,phillip.allen@enron.com,forecast,0
1,Traveling to have a business meeting takes the...,john.lavorato@enron.com,phillip.allen@enron.com,traveling business meeting take fun trip espec...,0
2,test successful. way to go!!!,leah.arsdall@enron.com,phillip.allen@enron.com,test successful way go,0
3,"Randy,Can you send me a schedule of the salary...",randall.gay@enron.com,phillip.allen@enron.com,randycan send schedule salary level everyone t...,0
5,"Greg,How about either next Tuesday or Thursday...",greg.piper@enron.com,phillip.allen@enron.com,greghow either next tuesday thursdayphillip,0


In [18]:

email_df.groupby('fraud').describe()

body                                                                 \
      count unique                                                top freq   
fraud                                                                        
0      9428   4717  george.huan@enron.com, mike.maggi@enron.com, l...   46   
1        36     21  jeffrey.hodge@enron.com, melissa.murphy@enron....    3   

         to                                                                 \
      count unique                                                top freq   
fraud                                                                        
0      9428   1585                            John Arnold/HOU/ECT@ECT  571   
1        36     16  aimee.shek@enron.com, albino.lopez@enron.com, ...    9   

      from_                                      body_clean         \
      count unique                     top  freq      count unique   
fraud                                                                
0      9428   1218   john.arnold@enron.com  2486       9428   4700   
1        36     14  outlook.team@enron.com     9         36     21   

                                                               
                                                     top freq  
fraud                                                          
0      georgehuanenroncom mikemaggienroncom larrymaye...   46  
1      jeffreyhodgeenroncom melissamurphyenroncomkyle...    3

In [20]:
! pip install tensorflow_text

  Using cached tensorflow_text-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.5 MB)
  Using cached tensorflow-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (524.1 MB)


In [19]:
email_df.head(5)

,body,to,from_,body_clean,fraud
0,Here is our forecast,tim.belden@enron.com,phillip.allen@enron.com,forecast,0
1,Traveling to have a business meeting takes the...,john.lavorato@enron.com,phillip.allen@enron.com,traveling business meeting take fun trip espec...,0
2,test successful. way to go!!!,leah.arsdall@enron.com,phillip.allen@enron.com,test successful way go,0
3,"Randy,Can you send me a schedule of the salary...",randall.gay@enron.com,phillip.allen@enron.com,randycan send schedule salary level everyone t...,0
5,"Greg,How about either next Tuesday or Thursday...",greg.piper@enron.com,phillip.allen@enron.com,greghow either next tuesday thursdayphillip,0


In [20]:
email_df.groupby('fraud').describe()

body                                                                 \
      count unique                                                top freq   
fraud                                                                        
0      9428   4717  george.huan@enron.com, mike.maggi@enron.com, l...   46   
1        36     21  jeffrey.hodge@enron.com, melissa.murphy@enron....    3   

         to                                                                 \
      count unique                                                top freq   
fraud                                                                        
0      9428   1585                            John Arnold/HOU/ECT@ECT  571   
1        36     16  aimee.shek@enron.com, albino.lopez@enron.com, ...    9   

      from_                                      body_clean         \
      count unique                     top  freq      count unique   
fraud                                                                
0      9428   1218   john.arnold@enron.com  2486       9428   4700   
1        36     14  outlook.team@enron.com     9         36     21   

                                                               
                                                     top freq  
fraud                                                          
0      georgehuanenroncom mikemaggienroncom larrymaye...   46  
1      jeffreyhodgeenroncom melissamurphyenroncomkyle...    3

## Splitting the dataset into training set and testing set over here

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(email_df['body_clean'],email_df['fraud'], stratify=email_df['fraud'])

In [22]:

y_train.value_counts()

0    7071
1      27
Name: fraud, dtype: int64

In [24]:
27/7071

0.003818413237165889

In [25]:
y_test.value_counts()

0    2357
1       9
Name: fraud, dtype: int64

In [26]:
9/2357

0.003818413237165889

## Installing the encoder and preprocessing model over here

In [27]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")


In [28]:
## pooling output is there for the entire sentence as we all know that
def get_sentence_embeding(sentences):
  preprocessed_text = bert_preprocess(sentences)
  return bert_encoder(preprocessed_text)['pooled_output']

In [29]:


get_sentence_embeding([
    "500$ discount. hurry up",
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.84351724, -0.5132727 , -0.88845736, ..., -0.7474883 ,
        -0.75314754,  0.91964495],
       [-0.87208354, -0.50543964, -0.94446665, ..., -0.85847497,
        -0.71745336,  0.88082975]], dtype=float32)>

In [30]:

e = get_sentence_embeding([
    "banana",
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "Manmeet Singh"
]
)

## Cosine similarity

In [31]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[1]])

array([[0.9911088]], dtype=float32)

In [32]:
# the two selected words are not same over here therefore similarity is not close to 99 in this case
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[5]])

array([[0.88983214]], dtype=float32)

## Building the BERT layers

In [33]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

## Building Neural Network Layers

In [34]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [35]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [36]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_word_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_type_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [37]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [40]:
model.fit(X_train, y_train, epochs=2)

Epoch 1/2
222/222 [==============================] - 88s 396ms/step - loss: 0.0226 - accuracy: 0.9962
Epoch 2/2
222/222 [==============================] - 84s 379ms/step - loss: 0.0223 - accuracy: 0.9962


In [41]:

model.evaluate(X_test, y_test)

74/74 [==============================] - 33s 422ms/step - loss: 0.0198 - accuracy: 0.9962


[0.019803380593657494, 0.9961960911750793]

In [42]:
reviews = ['''


hello world


    ''']

model.predict(reviews)

1/1 [==============================] - 1s 790ms/step


array([[0.00036852]], dtype=float32)

In [43]:

prediction = model.predict(reviews)

# Assuming it's a binary classification, the model's output is likely a probability.
# If you want to classify based on a threshold (0.5), you can do it like this:
if prediction > 0.5:
    print("fraud")
else:
    print("not fraud")

1/1 [==============================] - 0s 66ms/step
not fraud
